# Выбор оптимальной конфигурации модели

In [1]:
# standard libraries
import pandas as pd
import sys
import os
import yaml
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.float_format','{:_.2f}'.format)

# Выбор лучшей модели по метрикам по продукту

In [2]:
main_metrics = list(map(str.lower,['MedianAE', 'MAPE', 'MAE', 'MedianAE / Median','MAE / Mean']))
test_set_name = 'test'

def read_metric_table(file_name):
    table = pd.read_excel(file_name)
    table['config_model'] = file_name.split('.')[0].split('/')[1].replace('_metrics','')
    return table

# Загрзука метрик

In [3]:
# файлы с метриками по модели суммы кредита
metrics = sorted(filter(lambda x: 'model' in x, os.listdir('metrics')))
# соберем их в единый файл с указанием модели    
all_metrics = pd.concat(list(map(lambda x: read_metric_table('metrics/'+x),metrics))).reset_index(drop=True)

### Выбор лучшей конфигурации на test

In [4]:
oot = all_metrics.loc[all_metrics['dataset']=='test'].groupby(['group','measurement'])[main_metrics].idxmin().applymap(lambda x: all_metrics.iloc[int(x),-1])

In [5]:
oot.drop(columns=list(map(str.lower,['MedianAE / Median','MAE / Mean'])))

medianae  \
group                 measurement                                                                      
Инвестиционный кредит bin_principalapproved with bin = 0.0 from 520_0...  credit_sum_model_invest_v5   
                      bin_principalapproved with bin = 1.0 from 2_200...  credit_sum_model_invest_v5   
                      bin_principalapproved with bin = 2.0 from 4_445...  credit_sum_model_invest_v5   
                      bin_principalapproved with bin = 3.0 from 8_000...  credit_sum_model_invest_v5   
                      overall                                             credit_sum_model_invest_v5   
                      status with bin = Resolved-Completed                credit_sum_model_invest_v5   
                      status with bin = Resolved-Rejected                 credit_sum_model_invest_v5   

                                                                                                mape  \
group                 measurement                                                                      
Инвестиционный кредит bin_principalapproved with bin = 0.0 from 520_0...  credit_sum_model_invest_v5   
                      bin_principalapproved with bin = 1.0 from 2_200...  credit_sum_model_invest_v5   
                      bin_principalapproved with bin = 2.0 from 4_445...  credit_sum_model_invest_v5   
                      bin_principalapproved with bin = 3.0 from 8_000...  credit_sum_model_invest_v5   
                      overall                                             credit_sum_model_invest_v5   
                      status with bin = Resolved-Completed                credit_sum_model_invest_v5   
                      status with bin = Resolved-Rejected                 credit_sum_model_invest_v5   

                                                                                                 mae  
group                 measurement                                                                     
Инвестиционный кредит bin_principalapproved with bin = 0.0 from 520_0...  credit_sum_model_invest_v5  
                      bin_principalapproved with bin = 1.0 from 2_200...  credit_sum_model_invest_v5  
                      bin_principalapproved with bin = 2.0 from 4_445...  credit_sum_model_invest_v5  
                      bin_principalapproved with bin = 3.0 from 8_000...  credit_sum_model_invest_v5  
                      overall                                             credit_sum_model_invest_v5  
                      status with bin = Resolved-Completed                credit_sum_model_invest_v5  
                      status with bin = Resolved-Rejected                 credit_sum_model_invest_v5

### Выбор лучшей конфигурации на val

In [6]:
test = all_metrics.loc[all_metrics['dataset']=='val'].groupby(['group','measurement'])[main_metrics].idxmin().applymap(lambda x: all_metrics.iloc[int(x),-1])

In [7]:
test.drop(columns=list(map(str.lower,['MedianAE / Median','MAE / Mean'])))

medianae  \
group                 measurement                                                                      
Инвестиционный кредит bin_principalapproved with bin = 0.0 from 520_0...  credit_sum_model_invest_v5   
                      bin_principalapproved with bin = 1.0 from 2_200...  credit_sum_model_invest_v5   
                      bin_principalapproved with bin = 2.0 from 4_445...  credit_sum_model_invest_v5   
                      bin_principalapproved with bin = 3.0 from 8_000...  credit_sum_model_invest_v5   
                      overall                                             credit_sum_model_invest_v5   
                      status with bin = Resolved-Completed                credit_sum_model_invest_v5   
                      status with bin = Resolved-Rejected                 credit_sum_model_invest_v5   

                                                                                                mape  \
group                 measurement                                                                      
Инвестиционный кредит bin_principalapproved with bin = 0.0 from 520_0...  credit_sum_model_invest_v5   
                      bin_principalapproved with bin = 1.0 from 2_200...  credit_sum_model_invest_v5   
                      bin_principalapproved with bin = 2.0 from 4_445...  credit_sum_model_invest_v5   
                      bin_principalapproved with bin = 3.0 from 8_000...  credit_sum_model_invest_v5   
                      overall                                             credit_sum_model_invest_v5   
                      status with bin = Resolved-Completed                credit_sum_model_invest_v5   
                      status with bin = Resolved-Rejected                 credit_sum_model_invest_v5   

                                                                                                 mae  
group                 measurement                                                                     
Инвестиционный кредит bin_principalapproved with bin = 0.0 from 520_0...  credit_sum_model_invest_v5  
                      bin_principalapproved with bin = 1.0 from 2_200...  credit_sum_model_invest_v5  
                      bin_principalapproved with bin = 2.0 from 4_445...  credit_sum_model_invest_v5  
                      bin_principalapproved with bin = 3.0 from 8_000...  credit_sum_model_invest_v5  
                      overall                                             credit_sum_model_invest_v5  
                      status with bin = Resolved-Completed                credit_sum_model_invest_v5  
                      status with bin = Resolved-Rejected                 credit_sum_model_invest_v5

In [8]:
all_metrics.loc[(all_metrics['measurement']=='overall')&(all_metrics['config_model'].map(lambda x: int(x[-1]))==2),
                main_metrics+['config_model','group','dataset']].sort_values(['group','config_model','dataset'])

,medianae,mape,mae,medianae / median,mae / mean,config_model,group,dataset
